# Training Trails Notebook

This notebook contains all the code for training the models and hyperparameter tuning. It is split into different sections for different models.

In [1]:
# Import required libraries
import numpy as np
import pandas as pd


### Importing the dataset

In [2]:
data = pd.read_csv("../data/Dataset/Train.csv")
data.head()

,Date,Category,Brand,Day_of_Week,Holiday_Indicator,Past_Purchase_Trends,Price,Discount,Competitor_Price,Sales_Quantity
0,26/09/24,Automotive,BrandA,3,0,17.9,462.30,33.96,359.45,227.0
1,26/09/24,Automotive,BrandA,3,0,66.3,321.28,24.76,49.47,370.0
2,26/09/24,Automotive,BrandA,3,0,14.4,74.85,32.19,245.11,299.0
3,26/09/24,Automotive,BrandA,3,0,34.7,469.12,43.68,144.08,426.0
4,22/09/24,Automotive,BrandA,6,1,93.5,221.76,36.79,478.07,470.0


In [3]:
# Drop the rows having missing values
data.dropna(inplace=True)

## Date Preprocessing

In [4]:
data.head()

,Date,Category,Brand,Day_of_Week,Holiday_Indicator,Past_Purchase_Trends,Price,Discount,Competitor_Price,Sales_Quantity
0,26/09/24,Automotive,BrandA,3,0,17.9,462.30,33.96,359.45,227.0
1,26/09/24,Automotive,BrandA,3,0,66.3,321.28,24.76,49.47,370.0
2,26/09/24,Automotive,BrandA,3,0,14.4,74.85,32.19,245.11,299.0
3,26/09/24,Automotive,BrandA,3,0,34.7,469.12,43.68,144.08,426.0
4,22/09/24,Automotive,BrandA,6,1,93.5,221.76,36.79,478.07,470.0


In [5]:
# convert the date to datetime format and set as index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

C:\Users\amuly\AppData\Local\Temp\ipykernel_14376\3682652632.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Date'] = pd.to_datetime(data['Date'])


## Date Preprocessing

In [6]:
# Split the data into features and labels
X = data.drop("Sales_Quantity", axis=1)
y = data['Sales_Quantity']

In [7]:
X.head()

,Category,Brand,Day_of_Week,Holiday_Indicator,Past_Purchase_Trends,Price,Discount,Competitor_Price
Date,,,,,,,,
2024-09-26,Automotive,BrandA,3,0,17.9,462.30,33.96,359.45
2024-09-26,Automotive,BrandA,3,0,66.3,321.28,24.76,49.47
2024-09-26,Automotive,BrandA,3,0,14.4,74.85,32.19,245.11
2024-09-26,Automotive,BrandA,3,0,34.7,469.12,43.68,144.08
2024-09-22,Automotive,BrandA,6,1,93.5,221.76,36.79,478.07


In [8]:
y.head()

Date
2024-09-26    227.0
2024-09-26    370.0
2024-09-26    299.0
2024-09-26    426.0
2024-09-22    470.0
Name: Sales_Quantity, dtype: float64

In [9]:
split_size = int(0.8 * len(data))
X_train, y_train = X[:split_size], y[:split_size]
X_val, y_val = X[split_size:], y[split_size:]

In [10]:
# Get the shapes of train and val splits
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((14415, 8), (14415,), (3604, 8), (3604,))

#### Preprocessor Pipeline

In [11]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [12]:
# Define numerical transformer
numerical_transformer = Pipeline(
    steps = [
        ('scaler', StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps = [
        ('one_hot_encoder',OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ]
)

num_columns = list(X.select_dtypes(include=["number"]).columns)
cat_columns = list(X.select_dtypes(exclude=["number"]).columns)

preprocessor = ColumnTransformer(
    transformers = [
        ('num_transformer', numerical_transformer, num_columns),
        ('cat_transformer', categorical_transformer, cat_columns)
    ]
)

In [17]:
num_columns, cat_columns

(['Day_of_Week',
  'Holiday_Indicator',
  'Past_Purchase_Trends',
  'Price',
  'Discount',
  'Competitor_Price'],
 ['Category', 'Brand'])

In [13]:
# Transform the data
X_train_transformed = np.array(preprocessor.fit_transform(X_train))
X_val_transformed = np.array(preprocessor.transform(X_val))

## Modelling

In [14]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import root_mean_squared_error

In [15]:
# Create Dataset
class SalesDataset(Dataset):
    """ Dataset for Sales Quantity prediction """
    def __init__(self, X: np.ndarray, y: np.ndarray):
       super().__init__()
       self.X = X
       self.y = y
    
    def __len__(self):
       return len(self.X)
    
    def __getitem__(self, idx):
       return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.float32)

In [16]:
# Create DataLoaders

train_dataset = SalesDataset(X_train_transformed, y_train)
val_dataset = SalesDataset(X_val_transformed, y_val)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)